In [10]:
import pandas as pd

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('../01_preprocess/ycsb_e_init_new.csv')

#df.drop(['cycle', 'instr', 'LLC_miss','CPU','L1_miss', 'br_miss', 'scale', 'counted_final_key_count', 'GHz', 'task', 'IPC'], axis=1, inplace=True)
#df.drop(['nodeCount_Leaf', 'nodeCount_Inner', 'nodeCount_Dense', 'nodeCount_Hash', 'nodeCount_Head4', 'nodeCount_Head8', 'nodeCount_Dense2'], axis=1, inplace=True)

df.head()

,config_name,const_basicHintCount,const_enableBasicHead,const_enableDense,const_enableDense2,const_enableDensifySplit,const_enableHash,const_enableHashAdapt,const_enableHeadNode,const_enablePrefix,...,const_pageSizeLeaf,data_name,data_size,data_sorted,density,payload_size,rand_seed,ycsb_range_len,ycsb_zipf,time
0,dense3,16,1,1,0,1,0,0,0,1,...,8192,data/urls,4992374,0,0.815838,8,1706825744,100,1.254976,4.965867e-07
1,hints,16,1,0,0,0,0,0,0,1,...,2048,data/urls,2626244,0,0.795779,8,1706825754,100,0.593606,4.162030e-07
2,hash,16,1,0,0,0,1,0,0,1,...,4096,data/urls,2697613,0,0.722769,8,1706825798,100,0.782464,3.329775e-07
3,dense3,16,1,1,0,1,0,0,0,1,...,4096,data/urls,5182350,0,0.552645,8,1706825812,100,1.335837,4.798930e-07
4,hash,16,1,0,0,0,1,0,0,1,...,8192,data/urls,3904135,0,0.587902,8,1706825835,100,1.023279,3.448148e-07


In [11]:
# Identify non-numeric columns
non_numeric_columns = df.select_dtypes(['object']).columns

print("Number of columns:", df.shape[1])

# Apply one-hot encoding to non-numeric columns
df = pd.get_dummies(df, columns=non_numeric_columns)

print("Number of columns after one-hot:", df.shape[1])

Number of columns: 27
Number of columns after one-hot: 29


In [13]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Split the data into training and testing sets
X = df.drop('time', axis=1)
y = df['time']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Gradient Boosting Regressor
gb_regressor = GradientBoostingRegressor()
gb_regressor.fit(X_train, y_train)

# Predict on the test set
y_pred = gb_regressor.predict(X_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
r2 = r2_score(y_test, y_pred)
print("R-squared:", r2)


Mean Squared Error: 2.2139088955127893e-16
R-squared: 0.9227160473718662


OLD PART

In [4]:
category = pd.qcut(y_train, q=10, labels=False)
def find_category(number):
    
    closest_value = min(category, key=lambda x: abs(x - number))
    return closest_value


In [5]:
# Calculate the predicted category for each data point in the test set
predicted_category = [find_category(prediction) for prediction in y_pred]

# Calculate the actual category for each data point in the test set
cut = pd.qcut(df['time'], q=10, labels=False, duplicates='drop')

# Filter y_test to only include indices present in 'cut'
y_test_filtered = [actual for actual in y_test if actual in cut.index]

actual_category = [cut[actual] for actual in y_test_filtered]

# Calculate the precision
precision = sum([1 for pred, actual in zip(predicted_category, actual_category) if pred == actual]) / len(y_test_filtered)
print("Precision:", precision)


ZeroDivisionError: division by zero

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler 
scaler = StandardScaler() 
# Split the data into training and testing sets
X = df.drop('time', axis=1)
y = df['time']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler.fit(X_train) 
X_test = scaler.transform(X_test)  
# Train the Gradient Boosting Regressor
gb_regressor = GradientBoostingRegressor()
gb_regressor.fit(X_train, y_train)

# Predict on the test set
y_pred = gb_regressor.predict(X_test)

# Calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Mean Squared Error: 5.681879880096629e-15


/home/roland/.cache/pypoetry/virtualenvs/code-RswzgKQQ-py3.11/lib/python3.11/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
